#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [4]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/p_fc.csv')
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/es_interaction.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.649536,0.001000,0.649536
1,ABL1,AT13148,10.0,inc,0.591795,0.001000,0.591795
2,ABL1,AZ20,10.0,inc,0.652697,0.001000,0.652697
3,ABL1,AZD1480,10.0,inc,0.570660,0.001000,0.570660
4,ABL1,AZD3759,10.0,dec,0.837416,0.837416,0.001000
...,...,...,...,...,...,...,...
483,SRC,Torin,12.0,inc,0.775683,0.001000,0.775683
484,SRC,Trametinib,12.0,inc,0.736235,0.001000,0.736235
485,SRC,U73122,12.0,inc,0.551325,0.001000,0.551325
486,SRC,Ulixertinib,12.0,inc,0.602812,0.001000,0.602812


Splitting data into training/testing

In [5]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK', 'GSK690693', 'KD025', 'Vemurafenib', 'PF4708671', 'Tofacitinib', 'MK2206', 'GO6983', 'Dasatinib', 'AZD6738', 'PIK294', 'PD153035', 'KN62', 'Ulixertinib']


In [6]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(training_data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(training_data['p_fc'], 'phosphosite', 'sample', 'value')

In [7]:
samples = list(set(training_data['e_ksea']['sample']).union(set(training_data['p_fc']['sample'])))
phosphosites = list(set(training_data['p_fc']['phosphosite']))
evidence_dict = {}
for s in samples:
    evidence_dict[s] = {}
    for p in phosphosites:
        enzymes = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e]))
        evid_generator = build.ProblogStatementGenerator(predicates['p_fc'])
        evidence_p = evid_generator.generate_facts(build.EvidenceTemplate, select=[s, p])
        evidence_dict[s][p] = evidence_e + evidence_p

In [8]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/p_model/evidence_exp3.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

In [10]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/p_model/evidence_exp3_training.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

LFI

In [9]:
DIR = 'models/ebdt_data/sub_network/p_model/'
max_iter = 200
# learning from interpretation
cmd = f'problog lfi {DIR}p_model_lfi_exp3.pl {DIR}evidence_exp3.pl -O {DIR}p_model_exp3_200i.pl --logger {DIR}log_exp3-200i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

-504.41460283945065 [0.0, 1.0, 0.0, 0.0, 0.71699177, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.65344739, 1.0, 0.0, 0.99780701, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.96103101, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.20894543, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.66573594, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.1195131, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.12307457, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.2108109, 0.99996787, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.9988965, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.99998359, 1.0, 1.0, 1.0, 0.0, 0.32190935, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.56163327, 0.68360871, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.73046517, 

0

In [5]:
DIR = 'models/ebdt_data/sub_network/p_model/'
# learning from interpretation
cmd = f'problog ground {DIR}p_model_lfi_exp3.pl -o {DIR}p_model_exp3_ground.pl --format pl'
os.system(cmd)

(True, 't(0.33,"SRC","GO6983")::e_activity("SRC","GO6983",dec); t(0.33,"SRC","GO6983")::e_activity("SRC","GO6983",inc).\n0.001::e_ksea("SRC","GO6983",inc) :- e_activity("SRC","GO6983",dec).\n0.9106057458::e_ksea("SRC","GO6983",inc) :- e_activity("SRC","GO6983",inc).\n0.0446971271::e_ksea("SRC","GO6983",inc).\nt(0.33,"ABL1","GO6983")::e_activity("ABL1","GO6983",dec); t(0.33,"ABL1","GO6983")::e_activity("ABL1","GO6983",inc).\n0.001::e_ksea("ABL1","GO6983",inc) :- e_activity("ABL1","GO6983",dec).\n0.5072410486::e_ksea("ABL1","GO6983",inc) :- e_activity("ABL1","GO6983",inc).\n0.2463794757::e_ksea("ABL1","GO6983",inc).\nt(0.33,"PTK2","GO6983")::e_activity("PTK2","GO6983",dec); t(0.33,"PTK2","GO6983")::e_activity("PTK2","GO6983",inc).\n0.001::e_ksea("PTK2","GO6983",inc) :- e_activity("PTK2","GO6983",dec).\n0.5454588191::e_ksea("PTK2","GO6983",inc) :- e_activity("PTK2","GO6983",inc).\n0.2272705905::e_ksea("PTK2","GO6983",inc).\nt(0.33,"PTPRG","GO6983")::e_activity("PTPRG","GO6983",dec); t(0.3

256